In [1]:
library("phenopath")
library("reticulate")
np <- import("numpy")

I am not sure whether the CAVI optimization from phenopath is completely deterministic, so I'm not sure whether the following cell would have actually been necessary for complete reproducibility. Nevertheless, I should have included this in the original notebooks.

In [2]:
set.seed(42)

In [3]:
phenopath_defaults <- function(observations, covariates, ...) {
    # suppressWarnings because I am well aware that it hasn't converged
    return(suppressWarnings(phenopath(observations, covariates, model_mu=TRUE, 
                                      maxiter=50, thin=10, verbose=FALSE, ...)))
}

The gLV models have intercepts $\mu$, and it makes more sense conceptually that a model has them (so they can represent 'intrinsic growth rates'), so `model_mu` is being used even though it is not a default setting. Plus very preliminary investigation seemed to suggest it is likely to get better results, and one would expect that theoretically as well not just for the reasons above but also because the default `model_mu=FALSE` is actually a special case where $\mu = 0$ is assumed, so allowing extra possible values for that parameter in general can only increase the fit. (Of course one can then argue about overfitting, but that seems unlikely since this model is clearly heavily mis-specified anyway, and again the extra parameter would not only increase fit but also be amenable to biological interpretation in this context.)

Regarding small number of iterations, they seem to give decent enough results in practice, all the more so given that in this context one is more interested in qualitative fit, especially correct signs and sparsity, than precise numerical values. In other words any resulting quantitative accuracy will hopefully not also correspond to substantially decreased qualitative accuracy, especially given that 50 iterations is still a decent amount. Plus from a practical perspective, in order to provide estimates for so many estimations, there needs to be a reasonably enough upper bound on the runtime for each, and that is only possible by providing a relatively small of maximum possible iterations. In practice the experimenter, who only needs to analyze the results of a single experiment, should probably not used such a low number of maximum iterations.

In [4]:
spearman <- function(x,y){return(cor(x,y,method='spearman'))}

In [5]:
size = 11
seed = 42
number_droplets = 100000
number_batches = 5
results_dirname = 'results'

base_filename = paste(results_dirname, '/', size, '_strains.seed_', 
                      seed, '.', format(number_droplets, scientific=FALSE), '_droplets.iteration_',
                      '#', '.npz', sep='')

In [6]:
get_results <- function(phenopath_results, true_times) {
    uncensored_results <- interaction_effects(phenopath_results)
    censored_results <- significant_interactions(phenopath_results) * uncensored_results
    pearson <- abs(cor(true_times, trajectory(phenopath_results))) # same level of support if pseudotimes are flipped
    spearman <- abs(spearman(true_times, trajectory(phenopath_results)))
    
    results <- list("uncensored_results" = uncensored_results, 
                   "censored_results" = censored_results,
                   "pearson" = pearson,
                   "spearman" = spearman)
    return(results)
}

In [7]:
get_results_filename <- function(base_dir, scaling, iteration_number) {
    iteration_filename = paste('iteration_', iteration_number, '.npz', sep='')
    results_dir = file.path(paste(base_dir, '/', scaling, sep=''))
    results_filename = file.path(paste(results_dir, '/', iteration_filename, sep=''))
    return(results_filename)
}

save_results <- function(results_filename, results) {
    np$savez_compressed(results_filename,
    uncensored_results = results$uncensored_results,
    censored_results = results$censored_results,
    pearson = results$pearson,
    spearman = results$spearman)
}

The questions that I seek to answer are at a broad level: 'How can phenopath best be applied to this problem?' and 'What settings will allow phenopath to work best?'. Admittedly though not all possible choices of settings are being considered (e.g. alternative choices of `z_init`, like the true time values or the exponentials thereof, nor centering but not scaling the data, etc.) but the idea behind that was because in practice the experimenter is only likely to be willing (or in some cases only able) to use settings which do not deviate too much from the default values, plus it is reasonable to think that default choices of e.g. `z_init` were well thought out by the developers and are likely to be useful in practice, even if not the most useful theoretically possible for this context (which would be impossible to find, at least without a lot of probably intractable theory, due to e.g. the infinite state space of options for many settings).

So the specific questions reduce to: (1) does it make sense to scale the data in this context (even though the data is PCR-bias adjusted and so roughly absolute counts), (2) what makes the most sense to use as the covariates for the model, and (3) do censoring values considered "insignificant" by the phenopath model lead to increased fit (i.e. by reducing false positives)? As an additional fun question, it is also asked: does the correlation (Pearson and/or Spearman) of the computed pseudotimes with the true times of the batches have any predictive value for the performance? E.g. do the true time values have any "importance" in that sense?

As for always using the log-transformed data as the values, well that is both what is recommended in the phenopath paper and in its documentation, and moreover what makes the (generative) model for phenopath most closely resemble the (integral version of) the gLV equations.

create directories to store results in

In [8]:
all_results_dir = 'phenopath_results_cells'
if (!dir.exists(all_results_dir)) {dir.create(all_results_dir)}

binary_results_dir = file.path(paste(all_results_dir, '/', 'binary_covariates', sep=''))
if (!dir.exists(binary_results_dir)) {dir.create(binary_results_dir)}

counts_cov_results_dir = file.path(paste(all_results_dir, '/', 'count_covariates', sep=''))
if (!dir.exists(counts_cov_results_dir)) {dir.create(counts_cov_results_dir)}

for (subdirectory in list.files(path=all_results_dir, full.names=T)) {
    scaled_results_dir = file.path(paste(subdirectory, '/', 'scaled', sep=''))
    if (!dir.exists(scaled_results_dir)) {dir.create(scaled_results_dir)}
    
    unscaled_results_dir = file.path(paste(subdirectory, '/', 'unscaled', sep=''))
    if (!dir.exists(unscaled_results_dir)) {dir.create(unscaled_results_dir)}
}

loop through the iterations of stored results

In [9]:
# very bad naughty code that uses lots of global variables
# this is a self-contained notebook however and not a software library.
# I don't want to rewrite the function and the function calls to unnecessarily be
# much longer when scope rules automatically do the right thing.
lapply_input <- function(iteration_number){
    
    filename = gsub("#", iteration_number, base_filename)
    npzfile = np$load(filename)

    cell_log_counts = npzfile[["cell_log_counts"]]
    cell_init_vectors = npzfile[["cell_init_vectors"]]
    cell_counts = exp(cell_log_counts)*(cell_log_counts != 0)    
    
    merged_droplets_per_batch <- dim(cell_log_counts)[1]/number_batches
    true_times = c()
    for (i in 1:number_batches) {true_times <- append(true_times, rep(i, merged_droplets_per_batch))}
    
    start_time <- proc.time()
    binary_scaled <- phenopath_defaults(cell_log_counts, cell_init_vectors, scale_y=TRUE)
    results <- get_results(binary_scaled, true_times)
    save_results(get_results_filename(binary_results_dir, 'scaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    start_time <- proc.time()
    binary_unscaled <- phenopath_defaults(cell_log_counts, cell_init_vectors, scale_y=FALSE)
    results <- get_results(binary_unscaled, true_times)
    save_results(get_results_filename(binary_results_dir, 'unscaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    start_time <- proc.time()
    counts_scaled <- phenopath_defaults(cell_log_counts, cell_counts, scale_y=TRUE)
    results <- get_results(counts_scaled, true_times)
    save_results(get_results_filename(counts_cov_results_dir, 'scaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    start_time <- proc.time()
    counts_unscaled <- phenopath_defaults(cell_log_counts, cell_counts, scale_y=FALSE)
    results <- get_results(counts_unscaled, true_times)
    save_results(get_results_filename(counts_cov_results_dir, 'unscaled', iteration_number), results)
    run_time <- proc.time() - start_time; print(run_time)
    
    # this is supposed to be an embarrassingly parallel for loop, so memory usage should not change with number of iterations
    # but system monitor shows memory usage continually increasing. Hadley Wickham seems to have said that calling `gc`
    # manually for garbage collection should never be necessary, but honestly at this point I don't trust R so...
    gc()
    
    # controversial stylistically but consistent with Python style and since most programming is done
    # in Python, from a practical perspective it's better for me to use stylistic conventions that also work in Python.
    return(NULL)
}

In [10]:
lapply(1:100, lapply_input)

   user  system elapsed 
763.879  45.423 799.362 
   user  system elapsed 
767.960  53.491 818.546 
   user  system elapsed 
763.240  49.154 807.260 
   user  system elapsed 
761.953  50.214 808.497 
   user  system elapsed 
767.340  48.940 810.703 
   user  system elapsed 
789.185  56.694 850.560 
   user  system elapsed 
833.438  60.941 908.750 
   user  system elapsed 
822.821  59.499 890.906 
   user  system elapsed 
847.256  58.611 918.021 
   user  system elapsed 
837.767  63.880 916.148 
   user  system elapsed 
841.810  63.261 927.896 
   user  system elapsed 
845.689  62.658 929.492 
   user  system elapsed 
840.886  60.761 919.498 
   user  system elapsed 
843.240  63.757 926.766 
   user  system elapsed 
848.221  59.835 924.768 
   user  system elapsed 
839.952  65.650 924.552 
   user  system elapsed 
780.072  57.941 836.617 
   user  system elapsed 
777.086  56.547 831.377 
   user  system elapsed 
770.155  54.031 820.962 
   user  system elapsed 
767.591  52.214 818.025 


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
NULL

[[79]]
NULL

[[80]]
NULL

[[81]]
NULL

[[82]]
NULL

[[83]]
NULL

[[84]]
NULL

[[85]]
NULL

[[86]]
NULL

[[87]]
NULL

[[88]]
NULL

[[89]]
NULL

[[90]]
NULL

[[91]]
NULL

[[92]]
NULL

[[93]]
NULL

[[94]]
NULL

[[95]]
NULL

[[96]]
NULL

[[97]]
NULL

[[98]]
NULL

[[99]]
NULL

[[100]]
NULL